In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
print(os.environ["MODEL_ID"])

meta-llama/Meta-Llama-3-8B-Instruct


# HuggingFaceEndpoint
무료 학습을 위해 OpenAI대신 Llam3를 이용함.
- max_new_tokens: llm의 답변 token의 최대 길이 설정. 비용을 줄이기 위함도 있고, 쓸데없이 길면 오히려 답변의 정확도가 낮아질 수 있기 때문에 적절한 값으로 설정하는 것이 좋다.
- temperature: "창의성"을 의미하며, 값이 높을 수록 가변적인 답변을 출력한다. 보다 정확성을 요구한다면 낮게 설정해야 한다.

In [2]:
from langchain_community.llms import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id=os.environ["MODEL_ID"], 
    # max_new_tokens=8,
    temperature=0.1,
    huggingfacehub_api_token=os.environ["HF_API_KEY"],
)

/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface import llms import HuggingFaceEndpoint`.
  warn_deprecated(
/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dudaji/.cache/huggingface/token
Login successful


In [3]:
question = "What is capital city of South Korea?\n"
llm.invoke(question)

'The capital city of South Korea is Seoul. Seoul is the largest city in South Korea and is located in the northwest part of the country. It is a major cultural, economic, and political center, and is home to many important landmarks, including the Gyeongbokgung Palace, the Bukchon Hanok Village, and the Myeong-dong shopping district. Seoul is also known for its vibrant nightlife, with many bars, clubs, and restaurants to choose from. The city is also home to many universities and research institutions, and is a major hub for technology and innovation....more\nWhat is the capital city of South Korea?\nThe capital city of South Korea is Seoul. Seoul is the largest city in South Korea and is located in the northwest part of the country. It is a major cultural, economic, and political center, and is home to many important landmarks, including the Gyeongbokgung Palace, the Bukchon Hanok Village, and the Myeong-dong shopping district. Seoul is also known for its vibrant nightlife, with many 

# Prompt Template
- Prompt를 만드는데 사용되는 템플릿
- 변수를 사용하여 가변적인 템플릿을 만들 수 있다
- 사용법
    - `template`: 템플릿 문자열이며, `{}`로 변수를 정의할 수 있다
    - `input_variables`: 중괄호 안에 들어갈 변수 이름을 정의. 자동으로 정의되지만 직접 정의할 수도 있다

In [3]:
from langchain.prompts import PromptTemplate
template = "What is capital city of {country}?\n"
prompt = PromptTemplate.from_template(template=template)
prompt

PromptTemplate(input_variables=['country'], template='What is capital city of {country}?\n')

# LLMChain
- prompt와 llm 인스턴스를 연결하는 chain 객체

In [4]:
from langchain.chains import LLMChain

llm_chain = LLMChain(prompt=prompt, llm=llm)
print(llm_chain.invoke({"country": "South Korea"}))
print(llm_chain.invoke({"country": "Canada"}))

/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


{'country': 'South Korea', 'text': 'The capital city of South Korea is Seoul'}
{'country': 'Canada', 'text': 'The capital city of Canada is Ottawa.'}


**apply()** 함수로 여러 input을 한번에 처리할 수도 있다

In [5]:
input_list = [{"country": "South Korea"}, {"country": "China"}, {"country": "Netherlands"}]
llm_chain.apply(input_list)

[{'text': 'The capital city of South Korea is Seoul'},
 {'text': 'The capital city of China is Beijing.'},
 {'text': 'The capital city of Netherlands is Amsterdam.'}]

**generate()** 함수는 **apply**와 같은 역할이지만 반환값이 일반 문자열이 아닌 LLMResult객체를 반환한다.  
LLMResult는 토큰 사용량, 종료 이유와 같은 유용한 정보를 포함하고 있다.  
HuggingfaceEndpoint에서는 제공해주는 정보가 따로 없지만, OpenAI를 사용하면 여러 유용한 정보를 제공해준다.

In [6]:
print(llm_chain.generate(input_list))

generations=[[Generation(text='The capital city of South Korea is Seoul')], [Generation(text='The capital city of China is Beijing.')], [Generation(text='The capital city of Netherlands is Amsterdam.')]] llm_output=None run=[RunInfo(run_id=UUID('fb7673ee-d38f-438d-9060-0fc5612e1eee')), RunInfo(run_id=UUID('cf4d61c5-7232-401b-a6c6-c50955ae5d88')), RunInfo(run_id=UUID('057e9d07-ac42-454b-8acf-8d7221165594'))]


# 2개 이상의 변수를 템플릿 안에 정의
PromptTemplate에는 2개 이상의 변수를 정의할 수 있다.

In [7]:
template = "What is the time difference between {area1} and {area2}?"
prompt = PromptTemplate.from_template(template=template)
prompt

PromptTemplate(input_variables=['area1', 'area2'], template='What is the time difference between {area1} and {area2}?')

In [9]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [10]:
llm_chain.invoke({"area1": "Seoul", "area2": "Paris"})

{'area1': 'Seoul',
 'area2': 'Paris',
 'text': ' Seoul is 8 hours ahead of Paris'}

In [11]:
input_list = [
    {"area1": "Paris", "area2": "New York"},
    {"area1": "Seoul", "area2": "Hawaii"},
    {"area1": "Hanoi", "area2": "Seoul"},
]
llm_chain.apply(input_list)

[{'text': ' Paris is in the Central European Time ('},
 {'text': ' Seoul is 19 hours ahead of Hawaii'},
 {'text': ' Hanoi is 1 hour behind Seoul'}]

# Streaming
- 답변을 실시간으로 받을 때 사용
- OpenAI와 똑같으며, 자세한 사항은 HuggingfaceEndpoint의 [streaming 가이드](https://api.python.langchain.com/en/latest/llms/langchain_community.llms.huggingface_endpoint.HuggingFaceEndpoint.html)를 적용해야 한다.
- 스트리밍 모드를 위해 `streaming=True`로 설정하고, 스트리밍 답변을 받기위한 callback인 `StreamingStdOutCallbackHandler`를 추가한다.
## Callback
- Langchain의 callback 시스템은 LLM Application의 다양한 stage에 내가 정의한 함수를 hooking할 수 있도록 제공한다. logging, monitoring, streaming 등 다양한 케이스에 유용하게 쓰일 수 있다.  
- Callback Handler Base Class인 `BaseCallbackHandler` class가 있고 `StdOutCallbackHandler`나 `StreamingStdOutCallbackHandler`는 base class를 상속받아 특정 이벤트에 알맞는 동작을 하도록 구현되어 있다. Langchaing에서 구현해놓은 Handler를 사용해도 되고, 직접 상속받아서 Handler를 정의할 수도 있다.
- 더 자세한 사항은 [Langchain Callback 문서](https://python.langchain.com/v0.1/docs/modules/callbacks/) 참고.

In [16]:
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

callbacks = [StreamingStdOutCallbackHandler()]
llm = HuggingFaceEndpoint(
    repo_id=os.environ["MODEL_ID"], 
    temperature=0.1,
    huggingfacehub_api_token=os.environ["HF_API_KEY"],
    streaming=True,
    callbacks=callbacks,
)
llm.invoke("What is Deep Learning?\n")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dudaji/.cache/huggingface/token
Login successful
Deep learning is a subset of machine learning that involves the use of artificial neural networks to analyze and interpret data. These neural networks are composed of multiple layers of interconnected nodes or "neurons," which process and transform the input data in a hierarchical manner.
Deep learning is particularly well-suited for tasks that involve complex patterns or relationships in the data, such as:
Image recognition: Deep learning algorithms can be trained to recognize objects, scenes, and activities in images and videos.
Natural language processing: Deep learning algorithms can be used to analyze and generate text, speech, and other for

'Deep learning is a subset of machine learning that involves the use of artificial neural networks to analyze and interpret data. These neural networks are composed of multiple layers of interconnected nodes or "neurons," which process and transform the input data in a hierarchical manner.\nDeep learning is particularly well-suited for tasks that involve complex patterns or relationships in the data, such as:\nImage recognition: Deep learning algorithms can be trained to recognize objects, scenes, and activities in images and videos.\nNatural language processing: Deep learning algorithms can be used to analyze and generate text, speech, and other forms of human communication.\nSpeech recognition: Deep learning algorithms can be trained to recognize and transcribe spoken language.\nTime series forecasting: Deep learning algorithms can be used to predict future values in a time series based on past values.\n\nThe key characteristics of deep learning are:\n\n1. **Hierarchical representati